In [7]:
# Convert image stacks to zarr
import pathlib

import zarr
from ml4paleo.volume_providers import ImageStackVolumeProvider
import tqdm
from numcodecs import Blosc

In [14]:
image_path = pathlib.Path("/Users/mateljk1/Documents/resources/data/paleo/Pterosaur slices/Araripesaurus")
zarr_path = pathlib.Path("Araripesaurus.zarr", compressor=Blosc(cname='zstd', clevel=1, shuffle=Blosc.SHUFFLE))
image_stack = ImageStackVolumeProvider(image_path, "**/*.jp2")

In [15]:
image_stack[:,:,0].dtype, image_stack.shape

(dtype('uint16'), (3048, 2881, 9022))

In [ ]:
new_zarr_file = zarr.open(
    "araripesaurus.zarr",
    mode="w",
    shape=image_stack.shape,
    chunks=(256, 256, 256),
    dtype="uint16",
)

Z_CHUNK = 8
BACKGROUND_THRESHOLD = 6000

for z_start in tqdm.tqdm(range(0, image_stack.shape[2], Z_CHUNK)):
    stack = image_stack[:, :, z_start : z_start + Z_CHUNK]
    stack[stack < BACKGROUND_THRESHOLD] = 0
    new_zarr_file[:, :, z_start : z_start + Z_CHUNK] = stack
